In [1]:
import sympy as sp
import numpy as np

from src.py_utils.utils import tp, simplify, simplify_basis, simplify_mat, basis_disp
from src.py_utils.sos_sdp_maker import get_basis_matrix, get_constraint_tuples, output_julia
from src.py_utils.vars import w, i_to_w, w_to_i, A0, A1, B0, B1

In [2]:
from src.configs.B3 import basis, relations, game_poly, rounding_coeff
disp = lambda mat: basis_disp(mat, basis)
gp = sum([coeff * sp.expand(var[0] * var[1]) for coeff, var in game_poly])
basis_dag = sp.Matrix([O.adjoint() for O in basis]).T
basis

Matrix([
[         A0],
[         A1],
[      B0**2],
[      B1**2],
[      B0*B1],
[      B1*B0],
[      A0**2],
[      A1**2],
[         B0],
[         B1],
[      A0*B0],
[      A0*B1],
[A0**2*B0**2],
[A0**2*B1**2],
[      A1*B0],
[      A1*B1],
[A1**2*B0**2],
[A1**2*B1**2],
[          1]])

# Julia intermission

In [3]:
eqs = get_constraint_tuples(basis, relations)
output_julia(eqs, basis, game_poly, file_name='src/shared_data/input.jl')

[[[8, 18], [12, 1], [16, 2], [19, 4], [7, 14], [10, 19], [4, 10], [5, 9]],
 [[10, 8], [19, 18], [16, 19], [18, 16], [2, 4]],
 [[7, 3], [6, 12], [3, 11], [9, 1], [1, 13], [11, 7], [13, 9]],
 [[3, 5], [11, 12], [9, 10], [6, 4], [15, 16]],
 [[14, 11], [1, 6]],
 [[13, 1], [3, 7], [7, 11], [12, 6], [9, 13], [1, 9], [11, 3]],
 [[7, 13], [15, 2], [8, 17], [11, 1], [19, 3], [9, 19], [6, 10], [3, 9]],
 [[1, 15]],
 [[12, 15]],
 [[12, 11], [16, 15], [5, 3], [10, 9], [4, 6]],
 [[10, 14], [7, 12], [11, 5], [12, 4], [1, 10], [14, 1], [4, 7]],
 [[16, 8], [18, 10], [8, 4], [2, 18], [5, 15], [10, 2], [4, 16]],
 [[16, 17], [5, 2]],
 [[8, 13]],
 [[13, 11], [1, 3], [19, 13], [9, 7], [11, 19]],
 [[18, 8], [9, 5], [2, 16], [10, 4], [19, 10], [14, 7], [4, 19], [1, 12]],
 [[11, 9], [12, 10], [4, 14], [19, 7], [3, 13], [7, 1], [1, 19]],
 [[4, 2], [16, 18], [8, 10], [19, 16], [18, 19]],
 [[14, 9], [4, 11], [7, 6]],
 [[2, 1], [16, 12], [15, 11]],
 [[5, 8], [10, 17], [16, 3]],
 [[2, 19], [15, 9], [8, 2], [19, 8],

In [4]:
# import julia
# j = julia.Julia()
# M, game_value = j.include("optim.jl")

import os
os.system("julia src/julia_optim/optim.jl")

------------------------------------------------------------------
	       SCS v3.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 551, constraints m: 1656
cones: 	  z: primal zero / dual free vars: 915
	  s: psd vars: 741, ssize: 1
settings: eps_abs: 1.0e-10, eps_rel: 1.0e-10, eps_infeas: 1.0e-10
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 2561, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.41e+01  1.00e+00  2.68e+02 -1.34e+02  1.00e-01  6.96e-02 
   250| 1.24e-03  9.75e-06  1.21e-05  6.00e

0

# Python manipulation

In [5]:
from src.py_utils.clean_read import read_julia_mat, round_coeffs, clean_matrix

In [6]:
mat = read_julia_mat("src/shared_data/output.csv")
rc = rounding_coeff
mat = clean_matrix(mat.subs({sp.I: (2*w + 1)/np.sqrt(3)}), rc, eps=1e-4).subs(w_to_i)
G = (basis_dag * mat * basis)[0].simplify()
G = round_coeffs(simplify(G, relations))
game_value = simplify(G + gp.subs(w_to_i))
game_value

6

In [7]:
disp(mat)

Matrix([
[                            0,                        A0,                         A1,                        B0**2,                        B1**2,                        B0*B1,                        B1*B0, A0**2, A1**2, B0, B1,                        A0*B0,                        A0*B1,               A0**2*B0**2,               A0**2*B1**2,                       A1*B0,                       A1*B1,                A1**2*B0**2,                A1**2*B1**2,                          1],
[                  adjoint(A0),                      5/13,                          0,    -115/416 + 5*sqrt(3)*I/96,    -115/416 - 5*sqrt(3)*I/96,      5/208 + 5*sqrt(3)*I/312,      5/208 - 5*sqrt(3)*I/312,     0,     0,  0,  0,                            0,                            0,                         0,                         0,                           0,                           0,                          0,                          0,                          0],
[                  

In [8]:
import pickle 
with open('src/shared_data/SoS_mat.pkl', 'wb') as f:
    pickle.dump(mat, f)

# Getting into SoS Form

In [9]:
from src.SoS_utils.lu_utils import fast_symbolic_ldl
from src.SoS_utils.SoS import SoS_term, SoS_expr

In [10]:
l, d = fast_symbolic_ldl(mat)
sos = SoS_expr.from_Ucb(l.adjoint(), d, basis, relations)

In [12]:
assert simplify(sos.simplification + gp.subs(w_to_i) - game_value) == 0
with open("SoS.pkl", "wb") as f:
    pickle.dump(sos, f)